In [1]:
import numpy as np
import math
from scipy.optimize import curve_fit
from copy import deepcopy as dcopy
import matplotlib.pyplot as plt

---------------------------------------GEM 1 : ENCODER AZIMUTH CORRECTION-------------------------------------------

In [2]:
def main():
    
    #Coleta dados de input
    jlist, filelist, jset, jjset, numframes = user_inputs() #ok
    _, Dir1, _, _, Dir4 = directories() #ok
    tag, file_list, file_name_list,file_name = input_files(filelist) #ok
    n_files = len(file_list)
    
    # Para cada arquivo da filelist, faz as correções necessárias nos dados de azimut e
    # escreve os arquivos de output
    
    for file_name_index in range(n_files): 
        jframe, time, azimut = input_datarr(file_name_list[file_name_index],Dir1)#ok
        
        dpfazi,slope,dslope,j = azi_correction(jframe,azimut,time,numframes,tag,file_name_index,file_name)
        
        # Calcula e escreve nos arquivos de output as estatísticas relacionadas a rotação
        write_stat_rot(dpfazi,slope,dslope,j,file_name,Dir4)
        

In [3]:
def user_inputs():
    
    
    jlist = int(input("Single file (1) or Multiple files (2)?: \n"))
    print("\n")
    
    if jlist == 1:
        datetag = int(input("Enter the date of the year [yy_MM_dd] : \n"))
        jfilenum = int(input("Enter the UT time of the day [hhmmssml] : \n"))
        jj = 1
    else:
        filelist = input("ENTER the filelist name: \n")
        print("\n")
        jj = 0

    jset = int(input(" SET Vmax? [1] => yes / [2] => no \n"))
    print("\n")

    if jset == 1:
        Vmax = float(input(" Write Vmax: \n"))
    else:
        Vmax = 9.8993 # 1999 observations

    jjset = int(input("SET slp rate? [1] => yes / [2] => no \n"))
    print("\n")
    if jjset == 1:
        slp = float(input("Write slp rate: \n"))
    else:
        slp = 17.0357 # iterated 1999 Vmax +/- 0.3918

    numframes = int(input("ENTER the expected number of frames/scan sequence :[1994 > 106; 1999 > 105; 2005 > 103,104] \n"))
    
    return jlist, filelist, jset, jjset, numframes

In [4]:
def directories():
    
    " Diretórios."
    
    Dir = "/media/nicolli/eabdalla2012/resultados_nicolli/tradução_gem_modulos/Step_1_files/txt/" 
    Dir1 = "/media/nicolli/eabdalla2012/resultados_nicolli/tradução_gem_modulos/Step_1_files/dat/"
    Dir2 = "/media/nicolli/eabdalla2012/resultados_nicolli/tradução_gem_modulos/Step_2_Part1/rots/"
    Dir3 = "/media/nicolli/eabdalla2012/resultados_nicolli/tradução_gem_modulos/Step_2_Part1/azimuth/"
    Dir4 = "/media/nicolli/eabdalla2012/resultados_nicolli/tradução_gem_modulos/Step_2_Part1/rots/dpf/"
    
    return Dir, Dir1, Dir2, Dir3, Dir4

In [5]:
def input_files(filelist):
    
    " A partir das informações dadas pelo usuário, coleta os arquivos de input."

    Dir, Dir1, Dir2, Dir3, Dir4 = directories()
    with open(Dir + filelist) as arq: 
        filelist = arq.readlines()
        n_files = len(filelist)
        sub_file_list, datetag, tag, file_name_list = [], [], [], []
        for file_name_index in range(n_files):
            s_datetag = str(filelist[file_name_index])
            datetag.append(s_datetag[:8])
            tag.append(s_datetag)
            raw_name = filelist[file_name_index]
            sub_file_list.append(raw_name)
            #----------------------------
            file_name = raw_name
            file_name = file_name.replace("\t", "")
            file_name = file_name.replace("\n", "")
            file_name_list.append(file_name)
#             with open(Dir1 + file_name, "r+") as f: 
#                 input_data = f.readlines()
#                 n_lines = len(input_data)
        
    return tag, sub_file_list, file_name_list, file_name

In [6]:
def input_datarr(file_name,Dir1):
    
    " Coleta dados de input de cada um dos arquivos de uma determinada filelist."
 
    jframe, time, azimut = [],[],[]
    
    with open(Dir1 + file_name, "r+") as f: 
        input_data = f.readlines()
        n_lines = len(input_data)
        
    
    for line in input_data:
        input_line = line.replace(" ", "")
        input_line = line.replace("\n", "")
        input_line = input_line.split(" ")
        input_line = list(filter(None, input_line))
        jjframe = float(input_line[0])
        jtime = float(input_line[1])
        jazimut = float(input_line[4])
                
        jframe.append(jjframe)
        time.append(jtime)
        azimut.append(jazimut)
        
        #print("jframe",jframe)
        #print("time",time)
        #print("azimut",azimut)
        
    return jframe, time, azimut

In [7]:
def azi_correction(jframe,azimut,time,numframes,tag,file_name_index,file_name):
   
    " Para cada frame de um arquivo, verifica e corrige (com auxílio de outras funções) valores" 
    " de azimute incorretos."  
    
    print("Doing file:",tag[file_name_index])
    Dir,Dir1,Dir2,Dir3, Dir4 = directories()
    file_name = dcopy(tag[file_name_index])
    file_name = file_name.replace("\n","")
    
    timemean, aazmean, k, n_rotations, l, j105, j106, j = 0, 0, 0, 0, 0, 0, 0, 0
    
    aazimut, ttime, jccor, sig, dpfazi, speed, dspeed = [], [], [], [], [], [], []
    
    first, dangling, ic, new_rotation, first_time = True, False, None, False, True
    
    first_azi_new_rot = None
    
    i_frames = len(jframe)
    for i in range(i_frames): #for each line in file

        # Verficando até qual dado na lista de azimutes foi corrigido na rotação anterior.
        # A contagem de uma nova rotação deve iniciar no primeiro dado ainda não corrigido.
        # "first_azi_new_rot" é calculada no final de cada rotação, e indica onde a próxima
        # deve começar.
        
        if first_azi_new_rot is not None:
            if first_azi_new_rot != azimut[i]:
                first_azi_new_rot = None
                continue
                
        if first: # Get a reference in time and direction.
            compaz = azimut[i]   
            firstime = time[i]
            first = False
            
        #this is the expected behavior (azi-compaz<176), except for the very 
        #last azimut which may differ from the previous one
        #by < 176 and yet show reversal in sense of 
        #direction. The usual is a continuous decrease in 
        #azimut. An unexpected sign reversal can also happen. =>

        if (abs(azimut[i]-compaz)) < 176:
            l,ic,dangling,ttime,timemean,aazmean,sig,aazimut = azi_expected(i,105,aazimut,timemean,aazmean,\
            dangling,ttime,jccor,sig,time,azimut,compaz,l,ic,new_rotation)
            new_rotation = False
            #erro última rotação, fazer se i == 16500 e (abs(azimut[i]-compaz)) < 176, então vá p/
            #last_incpl_rotation
#             if i>=16499:
#                 print("file_name",file_name)
#                 print("aazimut",aazimut)
#                 print("l",l)
#                 print("\n\n")
            
            if i >= (i_frames - 1):
                l_incomplete_rot = dcopy(l)
                aazi_last_incomplete_rot = dcopy(aazimut)
                ttime_last_inc_rot = dcopy(ttime)
                time = dcopy(time)
                continue
                
        else:
            num_fr_per_rot = len(aazimut)
            if num_fr_per_rot >= (numframes-1):
                j,slope,dslope,speed,dspeed,lastazi,midazi,firstazi,dpfazi_in_this_fr,l2,dpfazi,last_corr_azi,\
                first_azi_new_rot = azi_signal_reverse(numframes,dangling,aazimut,\
                timemean,aazmean,ttime,azimut,l,i,ic,j,\
                dpfazi,speed,dspeed,j105,j106,first_time,jccor,time[i+1]) 
                
                first_time = False # Indica quando a função "azi_signal_reverse" deve iniciar a contagem
                # de dados que passam por ela. * Reinicia a cada rotação.
                
                # write in rot file (descrições dos arq. a escrever aq)
                dat_list_rot = [j,slope,dslope,lastazi,midazi,firstazi,dpfazi_in_this_fr]
                rot_type = "rot.dat"
                mode_rot = True
                aazi_dat = False
                write_output(dat_list_rot,file_name,Dir2,rot_type,mode_rot,aazi_dat)
            else:
                slope,dslope,aazi,l2,last_corr_azi,first_azi_new_rot = azi_incomplete_rotations(timemean,aazmean,ttime,time[i+1],aazimut,azimut,l,ic,i)
                l_incomplete_rot = dcopy(l)
                aazi_last_incomplete_rot = dcopy(aazimut)
                ttime_last_inc_rot = dcopy(ttime)
    
            aazimut,fpd = frame_p_degree(slope,l,l2,aazimut,ttime) 


            # write in azimut file
            dat_list_aazi = aazimut
            if (n_rotations) == 0: mode_aazi = True
            else: mode_aazi = False
            aazi_type = "azi.dat"
            aazi_dat = True
            write_output(dat_list_aazi,file_name,Dir3,aazi_type,mode_aazi,aazi_dat)
            
            # reset p/ próxima rotação 
            l,timemean,aazmean = 0,0,0 
            ic,dangling,new_rotation = None,False,True
            aazimut,ttime = [],[]
            n_rotations =+ 1
                
        if (abs(azimut[i]-compaz)) < 176:
            compaz = azimut[i] # atualiza o valor referência para o próx. azi dentro de uma rotação
        else:
            compaz = last_corr_azi  #atualiza o valor referência para a próxima rotação  
    
    # last incomplete rotation
    print("I",i)
    aazi_last_incomplete_rot = last_incpl_rotation(l_incomplete_rot,aazi_last_incomplete_rot,\
    fpd,ttime_last_inc_rot,time)
    # write last incomplete rotation
    dat_list_azi = dcopy(aazi_last_incomplete_rot)
    if (n_rotations) == 0: mode_azi = True
    else: mode_azi = False
    azi_type = "azi.dat"
    aazi_dat = True
    write_output(dat_list_azi,file_name,Dir3,azi_type,mode_azi,aazi_dat)
    
    return dpfazi,speed,dspeed,j 

In [8]:
def azi_expected(i,numframes,aazimut,timemean,aazmean,dangling,ttime,jccor,sig,time,azimut,compaz,l,ic,new_rotation):
    
    " Verifica se há uma reversão no sinal no último azimut (da rotação),"
    " ou se há uma inversão inesperada "
#     print(len(ttime), len(aazimut), "azi_expected")
    
    if i == 0 or new_rotation:
        l = 0
    else:
        l += 1
    
    if (l > 0) and (time[i] - ttime[l-1] < 0):
        ttime.append(time[i] + 86400)
    else:
        ttime.append(time[i])
        
    aazimut.append(azimut[i])
    timemean = timemean + ttime[l]
    aazmean = aazmean + azimut[i]

    sig.append(1)
    
    len_aazi = len(aazimut)
    
    if (azimut[i]-compaz) > 0: #
        if l >= (numframes-1):
            dangling = True #sign reversal in last azimut detected 
        else:
            if ic == None: 
                ic = 0 #unexpected sign
            else: 
                ic += 1
            jccor.append(l)

    return l,ic,dangling,ttime,timemean,aazmean,sig,aazimut

In [9]:
def azi_signal_reverse(numframes,dangling,aazimut,timemean,aazmean,ttime,\
                       azimut,l,i,ic,j,dpfazi,speed,dspeed,j105,j106,first_time,jccor,next_time):
    " Caso o valor atual de azi tenha decrescido fora do esperado (azi-compaz > 176),"
    " mas o número atual de 'l' contabilizados corresponde a uma rotação completa,"
    " então será feita uma correção nos valores de azi em que ocorreram reversões inesperadas,"
    " a partir do valor calculado de slope (vel./s) na função dpf."
    
    lastazi, midazi, firstazi= 0, 0, 0
    frame_rot_counter = len(aazimut)
                                              #In case the sign reversal had already occured
    if frame_rot_counter == (numframes-1):    #in the previous frame but the magnitude fell 
        j105 += 1                             #short of 176, we delete it from the mean (mm=1)
    if frame_rot_counter == numframes:        #double-dangling forces l to increase by 1. Even
        j106 += 1                             #when lastazi is really only the l-1 element, mid
                                              #azi serves just as well as firstazi to set dpfazi
    
    if first_time == True:
        j = 0  
    else: 
        j+=1 
        
    if dangling:# Se ocorreu reversão no último frame
#         print("m=1, esta passando aq")
        mm = 1
        slope,dslope,aazi,first_azi_new_rot,last_corr_azi = dpf(timemean,aazmean,ttime,next_time,aazimut,azimut[i],azimut[i+1],azimut[i+2],azimut[i+3],azimut,l,mm,i)
        lastazi = aazimut[l-2] 
        midazi = aazimut[l-1]
        firstazi = aazimut[l]
        l2 = 1
#         print("l",l)z
#         print("lastazi","midazi","firstazi",lastazi,midazi,firstazi)

    else: # Se não
#         print("m=0, esta passando aqui")
        lastazi = aazimut[l]
        mm = 0
        slope,dslope,aazi,first_azi_new_rot,last_corr_azi = dpf(timemean,aazmean,ttime,next_time,aazimut,azimut[i],azimut[i+1],azimut[i+2],azimut[i+3],azimut,l,mm,i)
        midazi = aazimut[l]
        firstazi = aazimut[l+1]
        l2 = 2
        
    speed.append(slope)
    dspeed.append(dslope)
    print("slope",slope)
    print("dslope",dslope)
    
    #If the Vmax used is correct, then midazi will be
    #spaced evenly from lastazi to firstazi according
    #to the prevailing slope. So we check and correct
    
    dpfazi_value = (360.0 + lastazi - firstazi)/2 # Calculando vel. graus/frame
    dpfazi.append(dpfazi_value)

    dpfazi_in_this_fr = (360.0 + lastazi - firstazi)/2
    
    # ic value = times unexpected signal reverse occurs (first time => ic = 0) 
    if not ic == None : 
        count = 0
        while count <= ic:
            aazimut[(jccor[count])] = aazimut[(jccor[count] - 1)] + (slope*0.56002)
            if aazimut[(jccor[count])] < 0: aazimut[(jccor[count])] += 360.0
            count += 1
#         print("count",count)
        
#     print("aazimut azi sig rev",aazimut)
#     print("jccor",jccor)
#     print("\n\n")
    
    return j,slope,dslope,speed,dspeed,lastazi,midazi,firstazi,dpfazi_in_this_fr,l2,dpfazi,last_corr_azi,first_azi_new_rot # write in rots file

In [10]:
def azi_incomplete_rotations(timemean,aazmean,ttime,next_time,aazimut,azimut,l,ic,i):
    
    " Caso 'l' corresponda a uma rotação incompleta, o valor de slope também será calculado. "
    
    print("Incomplete rotation of l frames =>",l)
    
    #print("timemean - azi_inc_rot_func",timemean)
    
    if not ic == None:
        mm = 1
        slope,dslope,aazi,first_azi_new_rot,last_corr_azi = dpf(timemean,aazmean,ttime,next_time,aazimut,azimut[i],azimut[i+1],azimut[i+2],azimut[i+3],azimut,l,mm,i)
#         print("azimut[i]",azimut[i])
#         print("slope azi_inc_rot",slope)
        l2 = 1
    else:
        mm = 0
        slope,dslope,aazi,first_azi_new_rot,last_corr_azi = dpf(timemean,aazmean,ttime,next_time,aazimut,azimut[i],azimut[i+1],azimut[i+2],azimut[i+3],azimut,l,mm,i)
        l2 = 2
    
    return slope,dslope,aazi,l2,last_corr_azi,first_azi_new_rot

In [11]:
def frame_p_degree(slope,l,l2,aazimut,ttime):
    
    " Calcula fpd (degree/frame), a partir do valor de slope, e realiza uma correção nos "
    " valores de azi que não forem compatíveis, numa margem, com o azimute calculado a partir de (vel*tempo)."
    
    fpd = slope*0.56002 
    
#     print("fpd",fpd)
#     print("len(ttime)",len(ttime))
#     print("l+l2",l+l2) 
#     print("l2",l2)  
#     print("ttime",ttime)

    d = 0
    while d <= (l+l2):
        if d >= (len(ttime)):
            ttime.append(0)
        if d > 0:
            aa_zi = abs(aazimut[d] - aazimut[d-1]) 
            tti_me = (ttime[d] - ttime[d-1])
            
            if (aa_zi < (abs(fpd/2))) or (aa_zi > (3*(abs(fpd/2)))) and (tti_me < (3*0.56002/2)):
                
                aazimut[d] = aazimut[d-1] + fpd 
        d += 1
    
#     print("aazimut fpd",aazimut)
#     print("\n\n")
    
    return aazimut,fpd

In [12]:
def last_incpl_rotation(l,aazimut,fpd,ttime,time):
    
    " Correção dos valores da última rotação."
    
#     print("l last_incpl_rot",l)
    print("aazimut function last incomplete rot",aazimut)
    print("\n\n")
#     print("n")
#     print("fpd function last incomplete rot",fpd)
#     print("ttime function last....",ttime)
#     print("t")
#     print("\n\n")
    
    i = 0
    while i <= l: #last incomplete rotation 
        if i > 0:
            
            if (abs(aazimut[i] - aazimut[i-1]) < abs(fpd/2)) or (abs(aazimut[i] - aazimut[i-1]) > abs(3*fpd/2))\
            and ((ttime[i] - ttime[i-1]) < (3*0.56002/2)):
                aazimut[i] = aazimut[i-1] + fpd
        i += 1
    
    print("aazimut function last incomplete rot - depois corr",aazimut)
    print("fpd",fpd)
        
    
    return aazimut

In [13]:
def write_output(listt, file_name, dirr, type_file, mode, aazi_dat):
    """
    Escreve os dados processados em seus respectivos arquivos.
    """
    
    if aazi_dat:
        new_name = file_name.replace(".dat", type_file)
        write_mode = "w" if mode else "a"  # Define o modo de escrita (sobrescrever ou anexar)
        
        if write_mode == "a":
            with open(dirr + new_name, "r") as arq_output:
                line = arq_output.readlines()[-1]
                line.replace("\n","")
                line.replace("\t","")
                last_item_last_rot = dcopy(float(line))
        else:
            last_item_last_rot = np.inf
                
        with open(dirr + new_name, write_mode) as arq_output:
            if abs(listt[0] - last_item_last_rot) > 0.01:
#                 print("lisst[0] e last item lats rot", listt[0],last_item_last_rot)
                arq_output.write(f"{listt[0]:.6f}\n")
            for i in range(1,len(listt)):  
                arq_output.write(f"{listt[i]:.6f}\n")  # Escreve cada item em uma linha, formatado com 2 casas decimais

    else:
        
        data = str(listt[0])
    #     print("listt",listt)
        for item in listt:
            item = f"{item:.2f}"
            data += f"\t{item}\t"
        new_name = file_name.replace(".dat", type_file)

        if mode:
            with open(dirr + new_name,"w") as arq_output:
                arq_output.write(data)
                arq_output.write("\n")
        else:
            print("dirr+new_name",dirr+new_name)
            with open(dirr + new_name,"a") as arq_output:
                arq_output.write(data)
                arq_output.write("\n")

In [14]:
def write_stat_rot(dpfazi,speed,dspeed,j,file_name,Dir4):
    
    slopmean,sigma,sigint,Z,dZ,dpfmean,sigdpf,fpd,sig_dpf,sigman_dpf,sigint_dpf,\
    mean_per,per_sigman,sigma_per,sigint_per,Vslp = stats_parameters(dpfazi,speed,dspeed,j)
    
    mode_rot_dpf = True
    listt_dpf = [fpd,sig_dpf,dpfmean,sigdpf,(fpd-dpfmean),Vslp]
    type_dpf = "dpf_rots.dat"
    dirr_dpf = Dir4
    aazi_dat = False
    write_output(listt_dpf,file_name,Dir4,type_dpf,mode_rot_dpf,aazi_dat)

In [15]:
#::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::				
def stats_parameters(dpfazi,speed,dspeed,j):
    """
    Calculates rotation statistics
    """
    Vmax, slp = 9.8993,17.0357 # for 1999 observations
    wt, slopemean, dpfmean, sigma, sigdpf = 0,0,0,0,0
    for i in range(j):
        w = 1.0/dspeed[i]**2
        slopemean += speed[i]*w
        wt += w 
        dpfmean += dpfazi[i]
    #print("w",w)
    slopemean = slopemean/wt #mean dish rotation speed in deg/sec
    dpfmean = dpfmean/j ##Average dpf speed in deg/frame
    for i in range(j):
        w = 1.0/dspeed[i]**2
        sigma = sigma + w*(speed[i] - slopemean)**2    
        sigdpf = sigdpf + (dpfazi[i] - dpfmean)**2

    sigma = math.sqrt(sigma/wt*j/(j-1)) # sigma dispersion (slopemean) deg/sec
    sigman = sigma/math.sqrt(j*1) # external error (slopemean) deg/sec
    sigint = math.sqrt(1/wt) # internal error (slopemean) deg/sec            
    Z = sigman/sigint # z statistic
    dZ = 1/math.sqrt(2*(j-1))         
    sigdpf = math.sqrt(sigdpf/(j-1))/math.sqrt(j*1)

    fpd = slopemean*0.56002 # frames/degree
    sig_dpf = sigma * 0.56002 # sigma dispersion
    sigman_dpf = sigman * 0.56002 # external error
    sigint_dpf = sigint * 0.56002 # internal error

    mean_per = 360/slopemean # Average rotation period in seconds
    per_sigman = 360*sigman/slopemean**2 # external error
    sigma_per = 360*sigma/slopemean**2 # sigma dispersion
    sigint_per = 360*sigint/slopemean**2 # internal error

    Vslp = Vmax + (fpd - dpfmean)/slp
    return(slopemean,sigma,sigint,Z,dZ,dpfmean,sigdpf,fpd,sig_dpf,sigman_dpf,sigint_dpf,mean_per,per_sigman,sigma_per,sigint_per,Vslp)

In [16]:
#::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
def dpf(tmean,zmean,ttim,next_time,aazi,azi,next_azi,first_azi_new_rot,m_azi,azimut,n,m,i):
    """
    Using the output of the least_sq function and the average values 
    of time and azimuth, calculate the speed in degrees/frame
    """

    if m == 0: #se não há inversão de sinal 
                            ##  tmean é inicialmente a soma de todos os valores de tempo 
        tmean = tmean/(n+1) ##  logo, aqui temos a média do tempo de uma rotação
        zmean = zmean/(n+1) ##  mesma coisa para o azimute
       
    else: #se há
        tmean = (tmean - ttim[n])/n
        zmean = (zmean - aazi[n])/n
        
    i = 0 ##
    while i <= n-m: ## subtrai valor médio em todos os dados, menos o último (se houver inversão)
        ttim[i] = (ttim[i] - tmean)
        aazi[i] = (aazi[i] - zmean)
        i += 1
    
    if ((n+1)-m) > 2:
        lsq_azi, lsq_tim = dcopy(aazi),dcopy(ttim)
        lsq_azi[-1],lsq_tim[-1] = 0, 0
        slope, zer, dslope, dzer  = least_sq(function, lsq_tim, lsq_azi) # ajuste linear(tempo,azimute)
    else:
        slope, dslope = 0, 0
    
    if m == 0:
        aazi.append(0)
        azi = next_azi
        next_azi = first_azi_new_rot
        first_azi_new_rot = m_azi
        aazi.append(azi)
    else:
        aazi.append(azi)
        
    ttim.append(next_time)
    
    i = 0
    while i <= n-m:
        aazi[i] += zmean
        i += 1
    
    jk = (n + 2) - m
    
    if (aazi[jk] <= next_azi) or (abs(aazi[jk] - next_azi) < (abs(slope * 0.56002 / 8 * 7))):
        dstep = (360 + aazi[n - m] - next_azi) / 3
        for i in range(1, 4):
            if i - m > 0:
                aazi.append(0)  # aazi é estendido para armazenar os valores
            aazi[n + i - m] = aazi[n - m] - i * dstep
            if aazi[n + i - m] < 0:
                aazi[n + i - m] += 360
    else:

        aazi[n + 1 - m] = aazi[n - m] - (360 + aazi[n - m] - aazi[n + 2 - m]) / 2
        if aazi[n + 1 - m] < 0:
            aazi[n + 1 - m] += 360

        if (n + 3 - m) >= len(aazi):
            aazi.append(0)  

        # Atualiza o valor corrigido
        aazi[n + 3 - m] = next_azi

    if aazi and aazi[-1] == 0:
        aazi.pop()
        
    last_corr_azi = dcopy(next_azi)
    first_azi_new_rot = dcopy(first_azi_new_rot)
#     print("first_azi_new_rot",first_azi_new_rot)
#     print("aazi array após correção - function dpf",aazi)
    print("\n\n")
    
    n += 1

    return slope,dslope,aazi,first_azi_new_rot,last_corr_azi

In [17]:
#:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::	
def least_sq(function,x,y):
    """
    Calculate a linear fit to the azimuth data as a function of time,
    obtaining the speed in degrees/sec and their respective uncertainties
    """
    x = np.array(x, dtype="f8")
    y = np.array(y, dtype="f8")
    param, m_cov = curve_fit(function, x, y) 
    slope, intercept = param

    perr = np.sqrt(np.diag(m_cov)) #siga, sigb
    
#     plt.scatter(x, y, color='blue', label='Dados')  
#     plt.plot(x, function(x, slope, intercept), color='red',label=f'Ajuste: y = {slope:.2f}x + {intercept:.2f}')  
#     plt.xlabel('Eixo X')
#     plt.ylabel('Eixo Y')
#     plt.legend()
#     plt.grid(True)
#     plt.show()
    
    return param[0],param[1],perr[0],perr[1]
#:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::

In [18]:
def function(x,a,b):
    return a*x + b  

#::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::

In [19]:
main()

Single file (1) or Multiple files (2)?: 
2


ENTER the filelist name: 
fileli001.txt


 SET Vmax? [1] => yes / [2] => no 
2


SET slp rate? [1] => yes / [2] => no 
2


ENTER the expected number of frames/scan sequence :[1994 > 106; 1999 > 105; 2005 > 103,104] 
105
Doing file: 99_05_1820234490.dat

Incomplete rotation of l frames => 59






slope -6.01841316440081
dslope 0.0002367014010512432



slope -6.018717847549299
dslope 0.00023678947653796233



slope -6.018402852436006
dslope 0.00024356554528031396



slope -6.018513588956412
dslope 0.00025384201667317673



slope -6.019023359024827
dslope 0.00022681442508058113



slope -6.019969521785526
dslope 0.00023298754186576597



slope -6.020498309290959
dslope 0.00020591973220431572



slope -6.020290855337309
dslope 0.00021320146631347882



slope -6.019713838777555
dslope 0.00023323644604116941



slope -6.019784236776413
dslope 0.0002308857647955994



slope -6.020095248028282
dslope 0.00023838752007323115



slope -6.0204557396748




slope -6.02104272876593
dslope 0.00022531030255481082



slope -6.021195981960791
dslope 0.0002500794132273681



slope -6.021356714295675
dslope 0.0002288734604297132



slope -6.021972768589401
dslope 0.00029762497484854355



slope -6.019788790670454
dslope 0.0002491592723787812



slope -6.020347863545325
dslope 0.00023009820852262376



slope -6.020316681530478
dslope 0.0002287375241468646



slope -6.020455116510317
dslope 0.00021274145890667112



slope -6.020483457617073
dslope 0.00023648247649344994



slope -6.020109653858953
dslope 0.00024062314710750454



slope -6.020431950893259
dslope 0.00022148662988637774



slope -6.020415230521034
dslope 0.0002198406179987708



slope -6.020437965331425
dslope 0.00021613206096953631



slope -6.020213749442433
dslope 0.0002248565125874896



slope -6.020292693662552
dslope 0.00021318064385140236
I 16499
aazimut function last incomplete rot [350.858, 347.518, 344.134, 340.817, 337.377, 334.037, 330.731, 327.291, 323.963, 320.568, 3




slope -6.020035655756906
dslope 0.00019901610362678992



slope -6.019929119106566
dslope 0.00019895695908051978



slope -6.01952991388816
dslope 0.00018988802597177297



slope -6.019577094338273
dslope 0.00021540195197612948



slope -6.019083288626699
dslope 0.00022891582005242326



slope -6.019808756466378
dslope 0.00019908484337923757



slope -6.019628211043568
dslope 0.0002078142581014954



slope -6.019531081872492
dslope 0.00020109352304896404



slope -6.019152032347241
dslope 0.00018099736680816998



slope -6.01988896329951
dslope 0.00021564188434989228



slope -6.019969915685772
dslope 0.0002171256324600456



slope -6.020369758277259
dslope 0.00020043950929996034



slope -6.02052953331903
dslope 0.0001958938820515914



slope -6.018414731041874
dslope 0.00024657208750665754



slope -6.019938099650006
dslope 0.0002022396155125332



slope -6.020020441223208
dslope 0.00021780596691707742



slope -6.020229501081971
dslope 0.00021674568504699375



slope -6.020048022